In [1]:
#pip install -q pyomo
#pip install gurobipy

import pyomo.environ as pyo

# sets (all caps)
PRODUCT = ['Purse', 'Wallet', 'Backpack']

RESOURCES = ['Leather', 'Sewing', 'Finishing']

# parameters ((descriptive names in lowercase letters))

revenue = {'Purse': 30, 'Wallet': 20, 'Backpack': 35}

daily_availability = {'Leather': 46, 'Sewing': 40, 'Finishing': 42}

resource_availability = {('Purse', 'Leather'): 4, ('Purse', 'Sewing'): 3, ('Purse', 'Finishing'): 5,
                           ('Wallet', 'Leather'): 2, ('Wallet', 'Sewing'): 1, ('Wallet', 'Finishing'): 2, 
                           ('Backpack', 'Leather'): 5, ('Backpack', 'Sewing'): 4, ('Backpack', 'Finishing'): 3}

# Model
model = pyo.ConcreteModel()

#Sensitivity imports
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
model.rc = pyo.Suffix(direction=pyo.Suffix.IMPORT)
model.slack = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Bounds
model.x = pyo.Var(PRODUCT, domain=pyo.NonNegativeReals)


#Objective
def obj_rule(model):
    return sum(model.x[i] * revenue[i]for i in PRODUCT)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

#Constraints
def availability(model, j):
    return (sum(resource_availability[i, j]*model.x[i] for i in PRODUCT) <= daily_availability[j])

model.availability_const = pyo.Constraint(RESOURCES, rule=availability)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Revenue: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in PRODUCT:
        print(f"{i}  = {pyo.value(model.x[i])}")
        
# print the sensitivity information    
print("Shadow Prices")
for c in model.component_objects(pyo.Constraint, active=True):
    print("   Constraint", c)
    for index in c:
        print("      ", index, model.dual[c[index]])
                   
print("Slacks")
for c in model.component_objects(pyo.Constraint, active=True):
    print("   Constraint", c)
    for index in c:
        print("      ", index, model.slack[c[index]])
        
print("Reduced Costs")
for c in model.component_objects(pyo.Var, active=True):
    print("   Var", c)
    for index in c:
        print("      ", index, model.rc[c[index]])
        


Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmpith01se1.pyomo.lp
Reading time = 0.00 seconds
x1: 3 rows, 3 columns, 9 nonzeros
Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x8f601701
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 5e+01]
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4500000e+32   6.875000e+30   1.450000e+02      0s
       3    4.3000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 